In [1]:
import os
os.chdir('../')
%pwd

'c:\\coding\\nlp\\Text-Summarizer'

In [2]:
'''
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
'''

'\n!pip install --upgrade accelerate\n!pip uninstall -y transformers accelerate\n!pip install transformers accelerate\n'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    gradient_accumulation_steps: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    save_strategy: str
    eval_steps: int
    save_steps: int
    save_total_limit: int
    load_best_model_at_end: bool

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import  read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_training
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            save_strategy = params.save_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            save_total_limit = params.save_total_limit,
            load_best_model_at_end = params.load_best_model_at_end, 
        )
        
        return model_trainer_config

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PegasusTokenizer, PegasusForConditionalGeneration
from datasets import load_dataset, load_from_disk
import torch

c:\Users\mnsha\miniconda3\envs\Text-Summarizer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-26 13:35:57,387: INFO: config: PyTorch version 2.5.1 available.]


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        # Set device to GPU if available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # logger.info(f"device = {device}")
        
        # Load the tokenizer and model
        tokenizer = PegasusTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = PegasusForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
        
        # Data collator for sequence-to-sequence models
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load the dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Tokenize datasets directly
        def preprocess_function(examples):
            return tokenizer(
                examples["dialogue"], 
                max_length=1024, 
                truncation=True, 
                padding="max_length"
            )

        tokenized_train = dataset_samsum_pt["test"].map(preprocess_function, batched=True)
        tokenized_validation = dataset_samsum_pt["validation"].map(preprocess_function, batched=True)
        
        # Set up training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.eval_strategy,
            save_strategy=self.config.save_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            save_total_limit=self.config.save_total_limit,
            load_best_model_at_end=self.config.load_best_model_at_end,
        )
        
        # Initialize the Trainer
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            data_collator=seq2seq_data_collator,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_validation,
        )
        
        # logger.info("Starting training...")
        
        # Train the model
        try:
            trainer.train()
        except Exception as e:
        #    logger.exception("Training failed.")
            raise e
        
        # Save the fine-tuned model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        
        # logger.info("Model training complete and saved.")

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-26 13:35:58,246: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-26 13:35:58,249: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-26 13:35:58,249: INFO: common: created directory at: artifacts]
[2024-12-26 13:35:58,253: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 818/818 [00:00<00:00, 1817.33 examples/s]
0it [00:00, ?it/s]
c:\Users\mnsha\miniconda3\envs\Text-Summarizer\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 0.0046, 'train_samples_per_second': 0.0, 'train_steps_per_second': 0.0, 'train_loss': 0.0, 'epoch': 0}
